# 1. Organização prévia

## 1.1 Módulos que serão utilizados
## 1.2 Funções implementadas

In [ ]:
#IMPORT MODULES
import os
import scipy
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
import operator
import xlwt
from xlwt.Workbook import *
from pandas import ExcelWriter
import xlsxwriter
import math as m
import matplotlib.cm as cm
from pandas.plotting import scatter_matrix
import itertools
from matplotlib import patches

In [ ]:
#Function 1: Obtaining the DataBase
def database(path, file):
    #Change directory
    os.chdir(path)
    #Data set
    db=pd.read_csv(file,sep=",",encoding="ISO-8859-1")
    #Data Matrix shape
    [l,c]=db.shape
    #Columns Header (Variables)
    header=db.dtypes.index
    return (db,l,c,header)

#Function 2.0: Three-Sigma Rule
def nsigma(var_orig, n=3):
    '''n -> multiplication factor of the threshold'''
    var=var_orig.copy()
    mean=var.mean(0)
    std=var.std(0)
    difference=abs(var-mean)
    threshold = n*std
    outlier_idx=difference>threshold
    var[outlier_idx]=np.nan
    return (var,outlier_idx,threshold)

#Function 2.1: Hampel Filter - moving window
def hampel(var_orig, rol='y', k=7, t0=3):
    '''vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)'''
    #Make copy so original not edited
    var=var_orig.copy()    
    #Hampel Filter
    L= 1.4826
    if rol=='y':
        rolling_median=var.rolling(k).median()#Median calculatade for each window, correspondent to each row -> the first (k-1) lines are set to NaN
        difference=np.abs(rolling_median-var)# The k first are valued as NaN
        median_abs_deviation=difference.rolling(k).median()
    else:
        median=var.median()#original Hampel
        difference=np.abs(median-var)
        median_abs_deviation= difference.median()
    threshold= t0 *L * median_abs_deviation
    outlier_idx=difference>threshold # NaN compered to anything always returns False -> the first k-1 values are never considered outlier
    var[outlier_idx]=np.nan
    return(var,outlier_idx,threshold)

#Function 3: Data Normalization
def scale(train,test,sk=1,rang=(0,1)):
    #sk=1 -> MinMaxScaler
    #sk=2 -> StandardScaler
    if sk==1:
        scaler=MinMaxScaler(feature_range=rang)
    else:
        scaler=StandardScaler()
    if len(train.shape)==1:
        tr=train.values.reshape(-1,1)
        te=test.values.reshape(-1,1)
        tr=np.reshape(scaler.fit_transform(tr),len(train))
        te=np.reshape(scaler.transform(te),len(test))
    else:
        tr=scaler.fit_transform(train)
        te=scaler.transform(test)
    return (tr,te,scaler)

#Function 4: Separate Database into input variables and output variables
def inout(data,header,output_number=1):
    #Output variables
    yname=header[-output_number]
    y=data[yname]
    #Input variables
    xname=header[:-output_number]
    x=data[xname]
    xs=x.shape
    ys=y.shape
    return(x,xname,xs,y,yname,ys)

# 2. Leitura da base de dados

## 2.1 Base de dados por completo
## 2.2 Base de dados apenas com as variáveis que serão efetivamente utilizadas

In [ ]:
#OBTAIN THE DATABASE
path="C:/Users/anab-/Documents/EQ Mestrado/A Pesquisa/Dados e Resultados/Caldeira de Recuperação"
file="db_20vc.csv"
db_orig,l,c,header_orig=database(path,file)
#The first and second columns are index and month -> should be removed
header=header_orig[2::]
dbb=db_orig.loc[:,header]
#Descriptive statistics
stat=dbb.describe()
header

In [ ]:
# CREATE A DIRECTORY TO SEND THE RESULTS
db=dbb.drop(['16-P_BD_KG/CM2'],axis=1)
dir = 'Results_new_outPBD'
if not os.path.isdir(dir): os.makedirs(dir)

figsize=(12,8)
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
db.describe()

In [ ]:
#UNIVARIATE SCATTER PLOTS
for i in range(db.shape[1]):
    plt.figure(figsize=(12,8))
    plt.plot(db.iloc[:,[i]],'kx',alpha=0.5)
    plt.xlabel('Observation')
    plt.ylabel(db.columns[i])
    plt.title(db.columns[i])
    fname = 'BeforeFilter-' + str(i+1) + '.png'
    plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
#EXCLUDE NOT USED VARIABLES
db0=db.drop(['17-C_H2S_PPM', '19-F_STREAM_T/H', '20-E_REDUC_%'],axis=1).replace(0,np.nan).dropna()
header0=db0.dtypes.index

#Data base after deletion of the SO2 disparate data
db01=db0.copy() #Unly the input variable included in the model
db01=db01.iloc[0:2395].append(db01.iloc[2545::])

db01.describe()

In [ ]:
#UNIVARIATE SCATTER PLOTS
'''plt.figure(figsize=(12,8))
plt.plot(db0.iloc[:,[-1]],'kx',alpha=0.5)
plt.xlabel('Observation')
plt.ylabel(header0[-1])
plt.title(header0[-1])
fname = 'SO2beforeremoval-' + str(i+1) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)
for i in range(db01.shape[1]):
    plt.figure(figsize=(12,8))
    plt.plot(db01.iloc[:,[i]],'kx',alpha=0.5)
    if i==16:
        plt.ylim((0,600))
    plt.xlabel('Observation')
    plt.ylabel(header0[i])
    plt.title(header0[i])
    
fname = 'SO2afterremoval-' + str(i+1) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)
'''

plt.figure(figsize=(12,8))
plt.plot(db0.iloc[2395:2545,[-1]],'kx',alpha=1,label='Dados Removidos')
plt.plot(db01.iloc[:,[-1]],'k.',alpha=0.1,label='Dados Mantidos')
plt.xlabel('Observação')#('Observation')
plt.ylabel('SO2 (ppm)')#(header0[-1])
plt.legend()
plt.title('SO2 (ppm) - antes e depois da remoção do conjunto discrepante')#(header0[-1])
fname = 'SO2beforeAfter-removal.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

plt.figure(figsize=(12,8))
plt.plot(db0.iloc[:,[-1]],'k-',alpha=1)
#plt.plot(db01.iloc[:,[-1]],'k.',alpha=0.1,label='Dados Mantidos')
plt.xlabel('Observação')#('Observation')
plt.ylabel('SO2 (ppm)')#(header0[-1])
plt.title('SO2 (ppm) - Dados completos')#(header0[-1])
fname = 'SO2complete.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
#SCATTER MATRIX  
fig=scatter_matrix(db01, alpha=0.2, figsize=(20, 20), diagonal='hist')
fname = 'Scatter Matrix - Original data'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)
#Histograms
plt.figure()
db01.hist(grid=False,figsize=(15,10))

In [ ]:
#CORRELATION MATRIX
cor=db0.corr() #Correlation Matrix
cor.style.background_gradient(cmap='PuBu')

# 3. PRÉ PROCESSAMENTO

## 3.1 Cálculo das médias das variáveis e verificação da coerência
## 3.2 Detecção de outlier

In [ ]:
# 3.1 SUBSTITUTION OF '3-P_FUEL-MMH20' and '4-P_FUEL-MMH20' BY THE AVERAGE
pfuel=db01.loc[:,('3-P_FUEL-MMH20','4-P_FUEL-MMH20')].copy()
#Mean of the values
meanfuel=(pfuel['3-P_FUEL-MMH20']+pfuel['4-P_FUEL-MMH20'])/2
pfuel['3.5-P_FUELmean-MMH2O']=meanfuel

#SCATTER MATRIX
scatter_matrix(pfuel, alpha=0.2, figsize=(8,8), diagonal='hist')
fname = 'Scatter-matrix p fuel.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)
#Correlation Matrix
corfuel=pfuel.corr() 
corfuel.style.background_gradient(cmap='PuBu')


In [ ]:
# 3.1 SUBSTITUTION OF '5-SS1_%' and '6-SS2_%' BY THE AVERAGE
ssolid=db01.loc[:,('5-SS1_%', '6-SS2_%')].copy()
#Mean of the values
meansolid=(ssolid['5-SS1_%']+ssolid['6-SS2_%'])/2
ssolid['5.5-SSmean_%']=meansolid

#SCATTER MATRIX
scatter_matrix(ssolid, alpha=0.2, figsize=(8, 8), diagonal='hist')
fname = 'Scatter-matrix solids.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)
#Correlation Matrix
corsolid=ssolid.corr() 
corsolid.style.background_gradient(cmap='PuBu')

In [ ]:
## 3.1 SUBSTITUTION OF THE TWO PRESSURES AND TWO SOLIDS CONTENTS MEASUREMENTS BY THEIR MEANS
db1 = db01.copy()
# Unification of '3-P_FUEL-MMH20' and '4-P_FUEL-MMH20' - Mean of pressures
Fm=(db1['3-P_FUEL-MMH20']+db1['4-P_FUEL-MMH20'])/2
#Modification of the database
db1['3-P_FUEL-MMH20']=Fm
db1.drop(['4-P_FUEL-MMH20'], axis=1, inplace=True)# Removing columns
db1.rename(columns={'3-P_FUEL-MMH20': '3.5-P_FUELmean-MMH2O'}, inplace=True)

# Unification of '5-SS1_%' and '6-SS2_%' - Mean Solids Content
SSm=(db1['5-SS1_%']+db1['6-SS2_%'])/2
#Modification of the database
db1['5-SS1_%']=SSm #Adding the %SS mean values to the new database
db1.drop(['6-SS2_%'], axis=1, inplace=True)# Removing columns
db1.rename(columns={'5-SS1_%': '5.5-SSmean_%'}, inplace=True)

head1= db1.dtypes.index
db1.shape

In [ ]:
# 3.1 CORRELATION MATRIX
cor1=db1.corr() #Correlation Matrix
cor1.style.background_gradient(cmap='PuBu')

In [ ]:
# 3.2 OUTLIER DETECTION
workfilter=db1.copy()
#Comparision of different filters
    #Those functions return the work database with Nan in place of Outliers and the index of the outliers
db_f1,out_idxf1,threshold1 = nsigma(workfilter,n=3) #Mean +- 3 sigma
workfilter=db1.copy()
db_f2,out_idxf2,threshold2 = hampel(workfilter,rol='n') #Just 1.4826*3*MAD
workfilter=db1.copy()
db_f3,out_idxf3,threshold3 = hampel(workfilter,rol='y') #Moving window
db_f1=db_f1.dropna()
db_f2=db_f2.dropna()
db_f3=db_f3.dropna()
print(np.sum(out_idxf1),db_f1.shape, np.sum(out_idxf2),db_f2.shape, np.sum(out_idxf3),db_f3.shape)

In [ ]:
# 3.2 CHART OUTLIER DETECTION

plt.figure(figsize=(12,8))
plt.plot(db1.iloc[:,-1],'k.',label='SO2 values')
plt.plot((0,db1.index.max()),(db1.iloc[:,-1].mean()-threshold1.iloc[-1],db1.iloc[:,-1].mean()-threshold1.iloc[-1]),'k--',label='3sigma threshold')
plt.plot((0,db1.index.max()),(db1.iloc[:,-1].mean()+threshold1.iloc[-1],db1.iloc[:,-1].mean()+threshold1.iloc[-1]),'k--')
plt.plot((0,db1.index.max()),(db1.iloc[:,-1].mean(),db1.iloc[:,-1].mean()),'k-',label='Mean')
plt.plot((0,db1.index.max()),(db1.iloc[:,-1].median()-threshold2.iloc[-1],db1.iloc[:,-1].median()-threshold2.iloc[-1]),'r:',label='Hampel threshold')
plt.plot((0,db1.index.max()),(db1.iloc[:,-1].median()+threshold2.iloc[-1],db1.iloc[:,-1].median()+threshold2.iloc[-1]),'r:')
plt.plot((0,db1.index.max()),(db1.iloc[:,-1].median(),db1.iloc[:,-1].median()),'r-',label='Median')
plt.legend()
#plt.ylim((0,200))#There is a outlier out there to change the mean
plt.xlabel('Sample')
plt.ylabel('SO2 (ppm)')
plt.title('Comparison 3sigma and Hampel-SO2')

fname = 'Comparison 3sigma and Hampel-SO2.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
# 3.2 Chosen Filter -> Hampel identifier
work=db_f2.copy()
head=work.dtypes.index
db_f2.describe()
threshold2#.iloc[-1]
2860-2108

In [ ]:
# 3.2 SCATTER MATRIX
scatter_matrix(work, alpha=0.2, figsize=(16, 16), diagonal='hist') 
plt.savefig(os.path.join(dir, 'Scatter-db-f2.png'), bbox_inches='tight', format='png', dpi=600)
#CORRELATION MATRIX
corw=work.corr() #Correlation Matrix
corw.style.background_gradient(cmap='PuBu')#'coolwarm')

In [ ]:
# 3.2 Univariate plots
for i in range(db1.shape[1]):
    plt.figure(figsize=(12,8))
    plt.plot(db1.iloc[:,i],'k--',alpha=0.5,label='Dados antes filtro')#Data before filtering
    plt.plot(work.iloc[:,i],'k-',alpha=1, label='Dados após filtro') #Data after filtering
    plt.legend()
    if i==db1.shape[1]-1:
        plt.ylim((25,250))
        plt.legend(loc='upper left')
    plt.xlabel('Observation')
    plt.ylabel(head1[i])
    plt.title(head1[i])
    fname = 'AfterFilter-' + str(i+1) + '.png'
    plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
db1.shape


# 3. Validação cruzada e Normalização
## 3.1 Validação cruzada
## 3.2 Normalização
## 3.3 Verificação dos resultados

In [ ]:
# 3.1 Separation of the rows with the highest and te lowest values of each variable, in order to ensure that they will be assigned to training set
kn=5
rs=42

#DATABASE SELECTED: 
data=work.copy()
head=work.dtypes.index
#Separation of the data: Input and Output
x,xname,xs,y,yname,ys=inout(data,head,output_number=1) #xs,ys = shapes of x and y

ind=[]
for i in range(x.shape[1]):
    ind.append(x[xname[i]].idxmax())
    ind.append(x[xname[i]].idxmin())
ind.append(y.idxmax())
ind.append(y.idxmin())
xminmax=x.loc[set(ind)]
xrest=x.drop(set(ind))
yminmax=y.loc[set(ind)]
yrest=y.drop(set(ind))

#Separation of the data into training set and testing set
ts= 0.2535#0.2005 #Testar 0.2535
xtr, xte, ytr, yte=train_test_split(xrest, yrest, test_size=ts, random_state=rs) 
xtr=xtr.append(xminmax)
ytr=ytr.append(yminmax)
xtr.shape,xtr.shape[0]/5, xte.shape[0]/data.shape[0], xte.shape[0]


In [ ]:
# 3.2 Normalização
#MinMaxScaler
        #For tanh: x and y => [-1,+1]
        #For logistic: x=>[-1,+1]; y=>[0,+1]
xtrain,xtest,scalerx=scale(xtr,xte,sk=1,rang=(-1,1))
ytrain,ytest,scalery=scale(ytr,yte,sk=1,rang=(-1,1))
ytrainl,ytestl,scaleryl=scale(ytr,yte,sk=1,rang=(0,1))
    #StandardScaler
xtrain1,xtest1,scalerx1=scale(xtr,xte,sk=2)
ytrain1,ytest1,scalery1=scale(ytr,yte,sk=2) #StandardScaler does not need a range definition
ytrai=ytrain.copy()
ytes=ytest.copy()


In [ ]:
# 3.3 Boxplot
medianprops = dict(linestyle=':', linewidth=2, color='k')
for j in range(x.shape[1]):
    plt.figure(figsize=(8,8))
    plt.boxplot([xtrain[:,j],xtest[:,j]],labels=['xtrain','xtest'], medianprops=medianprops)
    plt.title(head[j])
    fname = 'Boxplot-' + str(j+1) + '.png'
    plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)
plt.figure(figsize=(8,8))
plt.boxplot([ytrain,ytest],labels=['ytrain','ytest'], medianprops=medianprops)
plt.title(head[-1])
fname = 'Boxplot-' + str(15) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

# 4. Modelos de Regressão
## 4.1 Regressão Linear Múltipla


In [ ]:
# 4.1 Results from Multiple linear regression

n=0
kn=5 #number of splits at cross-validation procedure

#Writing the answer
column=['Y test','ŷ-cv1','ŷ-cv2', 'ŷ-cv3','ŷ-cv4','ŷ-cv5','ŷ-all','r2-cv1','r2-cv2','r2-cv3','r2-cv4','r2-cv5','r2-cvmean','r2-all']
mlr=pd.DataFrame(index=yte.index,columns=column).fillna(0)
mlr.iloc[:,0]=yte #Y value from database

# MULTIPLE LINEAR REGRESSION 
reg=sk.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
R2=np.zeros([1,kn])
#Cross-Validation
kf=sk.model_selection.KFold(n_splits=kn, shuffle=True, random_state=rs)
    #The training data is used for cross validation procedure. Te first testg set separeted is used after, for testing (https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)
    #Data set for cross validation: xtrain, ytrain
i=0
for train_index, test_index in kf.split(xtrain,ytrain):
    x_train, x_test = xtrain[train_index], xtrain[test_index]
    y_train, y_test = ytrain[train_index], ytrain[test_index]

    #Treining
    freg=reg.fit(x_train,y_train)
    parr=freg.get_params(deep=True)
    #Prediction
    y_hat=freg.predict(x_test) #Cross-validation prediction
    yhat=freg.predict(xtest) #Test Prediction

    #Return to original scale
    Y_hat=scalery.inverse_transform(y_hat.reshape(-1,1))
    Yhat=scalery.inverse_transform(yhat.reshape(-1,1))

    #Coefficient of Determination
    r2=freg.score(x_test,y_test) # Cross validation test set
    R2test=freg.score(xtest,ytest) #Real test set
    mlr.iloc[:,i+1]=Yhat #Crossvalidation
    mlr.iloc[:,i+7]=r2#R2test
    R2[0,i]=r2
    i+=1
#Treinar o modelo com todos os dados
freg=reg.fit(xtrain,ytrain)
parr=freg.get_params(deep=True)
#Prediction
yhat=freg.predict(xtest) #Test Prediction
Yhat=scalery.inverse_transform(yhat.reshape(-1,1)) #Return to original scale
R2test=freg.score(xtest,ytest) #Real test set
mlr.iloc[:,6]=Yhat
mlr.iloc[:,-2]=mlr.iloc[:,[7,8,9,10,11]].mean(axis=1)
mlr.iloc[:,-1]=R2test

stdev=mlr.iloc[:,[7,8,9,10,11]].std(axis=1).iloc[0]

mlr

In [ ]:
# 4.1 Results of MLR
import math as m
print(" r2_cv mean: ",mlr.iloc[0,-2],"\n","r2_complete data: ", mlr.iloc[0,-1])
print(" r_cv mean: ",m.sqrt(mlr.iloc[0,-2]),"\n","r_complete data: ", m.sqrt(mlr.iloc[0,-1]))
print(" residuals mean and stdev: ", (mlr.iloc[:,0]-mlr.iloc[:,6]).mean(0),(mlr.iloc[:,0]-mlr.iloc[:,6]).std(0))
print(" Maximum and Minimum error: ", abs(mlr.iloc[:,0]-mlr.iloc[:,6]).max(),abs(mlr.iloc[:,0]-mlr.iloc[:,6]).min())
print(" Mean absolute error: ",sum(abs(mlr.iloc[:,0]-mlr.iloc[:,6]))/mlr.iloc[:,6].shape[0])

#Coeficientes da regressão linear
print(freg.coef_ , freg.intercept_)
print(parr)
# 4.1 Plot results from multiple linear regression
plt.figure(3,figsize=(8,8))
plt.plot([75,115],[75,115],'-.', color='grey')
plt.plot(mlr.iloc[:,0],mlr.iloc[:,6],'k.') #r2 cv mean
plt.text(76,112,'R2 test: '+str(round(mlr.iloc[0,-1],4))+'\n'+'R test: '+str(round(np.sqrt(mlr.iloc[0,-1]),4)), fontsize=12)

plt.title('Multiple Linear Regression Avaliation: ŷ vs y')
plt.ylim((75,115))
plt.ylabel('ŷ (SO2 estimatives)')
plt.xlim((75,115))
plt.xlabel('ytest (SO2 real values)')
fname = 'MLR_ŷ-y' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

plt.figure(4,figsize=(8,8))
plt.hist(sorted(mlr.iloc[:,0]-mlr.iloc[:,6]),50, density=True, color='gray')
plt.xlabel('y-ŷ')
plt.ylabel('Probability density')
plt.title('Histogram of residuals - multiple linear regression'+'\n'+'Absolute Values')
plt.text(-16,0.085,'Residuals mean: '+str(round((mlr.iloc[:,0]-mlr.iloc[:,6]).mean(0),4))+'\n'+'Residuals standard deviation: '+str(round((mlr.iloc[:,0]-mlr.iloc[:,6]).std(0),4)), fontsize=10)
fname = 'MLR-Histogram of residuals-absolute' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

plt.figure(5,figsize=(8,8))
plt.plot([0,2600],[0,0],'-.', color='grey')
plt.plot((mlr.iloc[:,0]-mlr.iloc[:,6]),'k.')
plt.ylabel('y-ŷ')
plt.xlabel('Observation')
plt.title('Dispersion of the residuals - multiple linear regression'+'\n'+'Absolute Values')
fname = 'MLR-Dispersion of the residuals MLR-absolute' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)


plt.figure(6,figsize=(8,8))
plt.hist(sorted((mlr.iloc[:,0]-mlr.iloc[:,6])/mlr.iloc[:,0]),50, density=True, color='gray')
plt.xlabel('(y-ŷ)/y')
plt.ylabel('Probability density')
plt.title('Histogram of residuals - multiple linear regression'+'\n'+'Relative Values')
plt.text(-0.16,8.5,'Residuals mean: '+str(round(((mlr.iloc[:,0]-mlr.iloc[:,6])/mlr.iloc[:,0]).mean(0),4))+'\n'+'Residuals standard deviation: '+str(round(((mlr.iloc[:,0]-mlr.iloc[:,6])/mlr.iloc[:,0]).std(0),4)), fontsize=10)
fname = 'MLR-Histogram of residuals-relative' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

plt.figure(7,figsize=(8,8))
plt.plot([0,2600],[0,0],'-.', color='grey')
plt.plot((mlr.iloc[:,0]-mlr.iloc[:,6])/mlr.iloc[:,0],'k.')
plt.ylabel('(y-ŷ)/y')
plt.xlabel('Observation')
plt.title('Dispersion of the residuals - multiple linear regression'+'\n'+'Relative Values')
fname = 'MLR-Dispersion of the residuals MLR-relative' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
# MSE: Mean Squared Error
mse_mlr=sum((mlr.iloc[:,0]-mlr.iloc[:,6])*(mlr.iloc[:,0]-mlr.iloc[:,6]))/mlr.iloc[:,0].shape[0]
mse_mlr


## 4.2 Rede Perceptron em multicamadas

In [ ]:
## 4.2 Multilayer Perceptron
#Number of hidden layers variation
N=30 #maximal number of neurons at hidden layer
kn=5 #number of splits at cross-validation procedure
t=1e-3 #Tolerance (modifications at r2 - > training the net)
niter=7

#Neural Network parameters
n_neurons=list(range(1,N+1))
activs=['logistic', 'tanh']
solvs = ['lbfgs', 'sgd']
learn_rate= ['constant', 'invscaling', 'adaptive']
param_test=[] #MLP - Parameter names mapped to their values. For each test

#Writing the answer
cvn=ytr.shape[0]/5
ind1=np.arange(cvn)
ind2=np.array(yte.index)
column1=['ŷ-cv1','ŷ-cv2', 'ŷ-cv3','ŷ-cv4','ŷ-cv5','r2-cv1','r2-cv2','r2-cv3','r2-cv4','r2-cv5','r2_mean','r2_stdev']
column2=['Y_test','ŷ-test','r2-all','r2-cv1','r2-cv2','r2-cv3','r2-cv4','r2-cv5','r2_mean']
column3=['r-cv1','r-cv2','r-cv3','r-cv4','r-cv5','r_mean','r_stdev']
idx = pd.IndexSlice
index1= pd.MultiIndex.from_product([n_neurons, activs, solvs, learn_rate,ind1], 
                          names=['ActivFunc','Solver','LearnRate','NumberNeurons','Observation'])
index2= pd.MultiIndex.from_product([n_neurons, activs, solvs, learn_rate, ind2], 
                          names=['ActivFunc','Solver','LearnRate','NumberNeurons','Observation'])
mlp_cv=pd.DataFrame(index=index1,columns=column1)#Results from crossvalidation procedure
mlp_test=pd.DataFrame(index=index2,columns=column2)#Results all training set and test set (reidentificação da rede)
r_coef=pd.DataFrame(index=index1,columns=column3)#Results Correlation coefficient(r), crossvalidation

for m in range(N):#N #Neurons number
    m=m+1
    for h in range(2):#2 #activation
        if activs[h]=='logistic':
            ytrai=ytrainl.copy()
            ytes=ytestl.copy()
        else:
            ytrai=ytrain.copy()
            ytes=ytest.copy()

        for j in range(2):#2 #solver
            for k in range(3):#3 #learning rate
                nn=MLPRegressor(hidden_layer_sizes=(m+1,),
                           activation=activs[h], 
                           solver=solvs[j],
                           batch_size='auto',
                           learning_rate= learn_rate[k],
                           max_iter=1000,
                           random_state=rs,
                           tol=t,
                           n_iter_no_change=niter,
                           verbose=False)
                #https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
                kf=sk.model_selection.KFold(n_splits=kn, shuffle=True, random_state=rs)
                #The training data is used for cross validation procedure. The first testset sorted is used after, for testing (https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)
                #Data set for cross validation: xtrain, ytrain
                i=0
                for train_index, test_index in kf.split(xtrain,ytrai):
                    x_train, x_test = xtrain[train_index], xtrain[test_index]
                    y_train, y_test = ytrai[train_index], ytrai[test_index]

                    #Training
                    rb=nn.fit(x_train,y_train)
                    par=rb.get_params(deep=True)
                    #Prediction
                    y_hat=nn.predict(x_test)
                    #Test with the extra testing set - For each cross validation
                    yhat=nn.predict(xtest)
                    #Reshaping
                    y_h=y_hat.reshape(-1,1)
                    yh=yhat.reshape(-1,1)
                    if activs[h]=='logistic' or activs[h]=='relu':
                        Y_hat=scaleryl.inverse_transform(y_h)
                        Yhat=scaleryl.inverse_transform(yh)
                        y_t=scaleryl.inverse_transform(y_test.reshape(-1,1))
                    else:
                        Y_hat=scalery.inverse_transform(y_h)
                        Yhat=scalery.inverse_transform(yh)
                        y_t=scalery.inverse_transform(y_test.reshape(-1,1))
                    r2=rb.score(x_test,y_test)
                    R2=rb.score(xtest,ytes)

                    mlp_cv.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column1[i]]=Y_hat
                    mlp_cv.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column1[i+5]]=r2
                    r_coef.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column3[i]]=np.sqrt(r2)
                    mlp_test.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column2[i+3]]=R2
                    i+=1

                #Training with all data
                rb=nn.fit(xtrain,ytrai)
                #Test with the extra testing set - For each cross validation
                yhat=nn.predict(xtest)
                yh=yhat.reshape(-1,1)
                R2=rb.score(xtest,ytes)
                if activs[h]=='logistic': 
                    Yhat=scaleryl.inverse_transform(yh)
                else:
                    Yhat=scalery.inverse_transform(yh)
                #Writing the answer
                mlp_test.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column2[0]]=yte.values.reshape(-1,1)
                mlp_test.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column2[1]]=Yhat
                mlp_test.loc[idx[m,activs[h], solvs[j],learn_rate[k],:],column2[2]]=R2
                param_test.append([m,activs[h],solvs[j],learn_rate[k],nn.get_params()])


In [ ]:
param_test

In [ ]:
# 4.2 MEAN OF R2 FROM CROSSVALIDATION (Determination coefficient)
mlp_cv.loc[idx[:],column1[-2]]=mlp_cv.iloc[:,[5,6,7,8,9]].mean(axis=1)
mlp_cv.loc[idx[:],column1[-1]]=mlp_cv.iloc[:,[5,6,7,8,9]].std(axis=1)
mlp_test.loc[idx[:],column2[-1]]=mlp_test.iloc[:,[2,3,4,5,6]].mean(axis=1)

In [ ]:
# 4.2 MEAN OF R FROM CROSSVALIDATION (correlation coefficient)
r_coef.loc[idx[:],column3[-2]]=r_coef.iloc[:,[0,1,2,3,4]].mean(axis=1)
r_coef.loc[idx[:],column3[-1]]=r_coef.iloc[:,[0,1,2,3,4]].std(axis=1)
r_coef

In [ ]:
##CROSS VALIDATION
# 4.2 Take the better results per activation function for each neuron number at hidden layer
best_log=np.zeros((N,6)).astype(object)
best_tanh=np.zeros((N,6)).astype(object)
for i in range(N):
    best_log[i,:5]=r_coef.loc[idx[i+1,'logistic',:,:,:]].sort_values(['r_mean'], ascending=[False]).iloc[0].name
    best_log[i,-2]=r_coef.loc[idx[i+1,'logistic',:,:,:]].sort_values(['r_mean'], ascending=[False]).iloc[0,-2] #r_mean
    best_log[i,-1]=r_coef.loc[idx[i+1,'logistic',:,:,:]].sort_values(['r_mean'], ascending=[False]).iloc[0,-1] #r_stdev
    best_tanh[i,:5]=r_coef.loc[idx[i+1,'tanh',:,:,:]].sort_values(['r_mean'], ascending=[False]).iloc[0].name
    best_tanh[i,-2]=r_coef.loc[idx[i+1,'tanh',:,:,:]].sort_values(['r_mean'], ascending=[False]).iloc[0,-2] #r_mean
    best_tanh[i,-1]=r_coef.loc[idx[i+1,'tanh',:,:,:]].sort_values(['r_mean'], ascending=[False]).iloc[0,-1] #r_stdev
best_log, best_tanh

In [ ]:
# 4.2 Plot best logistic results and best tanh results - cross validation
xaxis=np.arange(1,31)
plt.figure(1,figsize)
plt.plot(xaxis,best_log[:,-2],'k.',label='logistic')
plt.errorbar(xaxis, best_log[:,-2],best_log[:,-1],linewidth=0,elinewidth=0.5, color='grey')
plt.plot(xaxis,best_tanh[:,-2],'kx',label='tanh')
plt.errorbar(xaxis, best_tanh[:,-2],best_tanh[:,-1],linewidth=0,elinewidth=0.5, color='grey')
plt.ylim(0.68,1.0)
plt.xlabel('Neurons at hidden layer')
plt.ylabel('r mean')
plt.legend()
plt.title('Cross validation corelation coefficient mean and errors for each activation function'+"\n" +'according to the number of units at hidden layer')
fname = 'r logistic and tanh' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
for i in range(N-1):
    print(i+2, 100*(best_tanh[i+1,-2]-best_tanh[i,-2])/best_tanh[i,-2])

In [ ]:
#plt.rcParams.update({'font.size': 16})
#matplotlib.rc('xtick', labelsize=12) 
#matplotlib.rc('ytick', labelsize=12) 
# 4.2 Plot restrictly tanh, lbfgs, constant - Cross validation
plt.figure(1,figsize)
for i in range(1,N+1):
    plt.errorbar(i,r_coef.loc[idx[i,'tanh','lbfgs','constant']].iloc[0,-2],yerr=r_coef.loc[idx[i,'tanh','lbfgs','constant']].iloc[0,-1],elinewidth=0.5, ecolor='black')#Mean of R2 of the crossvalidation regression
    plt.plot(i,r_coef.loc[idx[i,'tanh','lbfgs','constant']].iloc[0,-2],'k.')
plt.text(8.,0.938,9)
plt.arrow(8.5,0.938,0.23,-0.007)
plt.text(11.,0.94,12)
plt.arrow(11.5,0.938,0.23,-0.007)

plt.plot(xaxis,best_log[:,-2],'k.',label='logistic')
plt.errorbar(xaxis, best_log[:,-2],best_log[:,-1],linewidth=0,elinewidth=0.5, color='grey')

plt.title('Mean and standard deviation: r mean of cross-validation (actv function: tanh)')
plt.ylabel('r mean')
plt.xlabel('Number of neurons at hidden layer')
fname = 'r mean of cross-validation x nn_logtanh.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

# Performance improvement provided by addition of units at hidden layer
bla=[]
xs = np.arange(2,N+1,1)

fig = plt.figure(2,figsize)
ax = fig.add_subplot(111)
for i in range(N-1):
    dif=(best_tanh[i+1,-2]-best_tanh[i,-2])/best_tanh[i,-2]#Relative increase
    bla.append(100*dif)
    plt.bar(i+2,100*dif, color='gray')
    plt.xticks(np.arange(2, 31, 1))

e1 = patches.Ellipse(xy=(9,1.15), width=1.4,height=0.45, fill=False)
ax.add_patch(e1)
e = patches.Ellipse(xy=(12,1.3), width=1.4,height=0.45, fill=False)
ax.add_patch(e)

ax.set_title('Performance increase by adding units at hidden layer')
ax.set_ylabel('Percentage increase at r mean')
ax.set_xlabel('Comparisons of unit_(i) and unit_(i-1)')

for x,y in zip(xs,bla):
    if y < 0:
        space = -10
    else:
        space = 8
    label = "{:.2f}".format(y)
    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,space), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
fname = 'Percentage increase at r mean-unit-unit' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
best_test=pd.DataFrame(columns=mlp_test.columns)
for i in range(1,N+1):
    best_test.loc[i] = mlp_test.loc[idx[i,'tanh','lbfgs','constant']].iloc[0,:]
best_test['r2_stdev']=best_test.iloc[:,3:8].std(axis=1)

# 4.2 Plot restrictly tanh, lbfgs, constant - Cross validation
plt.figure(1,figsize)
for i in range(1,N+1):
    plt.errorbar(i,best_test.iloc[i-1,-2],yerr=best_test.iloc[i-1,-1],elinewidth=0.5, ecolor='black')#Mean of R2 of the crossvalidation regression
    plt.plot(i,best_test.iloc[i-1,-2],'k.')
    if i==9:
        plt.plot(i,best_test.iloc[i-1,-2],'rx')  
plt.title('Mean and standard deviation: r2 mean of test set (actv function: tanh)')
plt.ylabel('r2 mean')
plt.xlabel('Number of neurons at hidden layer')
fname = 'r2 mean of test set x nn' + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
neurons=9

# 4.2 Plot y vs ŷ -> TEST SET
plt.figure(1,figsize=(8,8))
plt.plot([73,120],[73,120],'-.', color='grey')
plt.plot(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0],mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1],'k.')
plt.xlabel('Y_test (SO2 real values)')
plt.ylabel('Ŷ_test (SO2 estimatives)')
plt.ylim((73,120))
plt.xlim((73,120))
plt.title('MLP regression avaliation: ŷ vs y '+"\n (" +str(neurons)+' units at hidden layer and tanh as activation function)')
plt.text(75,115,'R2 test: '+str(round(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[0,2],4))+'\n'+'R test: '+str(round(np.sqrt(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[0,2]),4)), fontsize=12)
fname = 'MLP-Regression_y vs ŷ ' +str(neurons)+ '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Histogram -> y-ŷ -> ABSOLUTE VALUES

dify=(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])
plt.figure(2,figsize=(8,8))
plt.hist(sorted(dify),50, density=True, color='gray')
plt.title('Histogram of residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Absolute values')
plt.xlabel('y-ŷ')
plt.ylabel('Probability density')
plt.text(-14,0.175,'Residuals mean: '+str(round(dify.mean(0),4))+'\n'+'Residuals standard deviation: '+str(round(dify.std(0),4)), fontsize=11)

fname = 'MLP-Histogram of residuals_y-ŷ_absolute-'+str(neurons) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Residuals -> ABSOLUTE VALUES
plt.figure(3,figsize=(8,8))
plt.plot([0,2600],[0,0],'-.', color='grey')
plt.plot(dify,'k.')
plt.ylabel('y-ŷ')
plt.xlabel('Observation')
plt.ylim((-13,13))
plt.title('Dispersion of the residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Absolute values')
fname = 'MLP-Dispersion of the residuals MLP_absolute-' +str(neurons)+ '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Histogram -> y-ŷ -> RELATIVE VALUES

dify=(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])
difyrel=dify/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]
plt.figure(4,figsize=(8,8))
plt.hist(sorted(difyrel),50, density=True, color='gray')
plt.title('Histogram of residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Relative values')
plt.xlabel('(y-ŷ)/y')
plt.ylabel('Probability density')
plt.text(-0.15,17.5,'Residuals mean: '+str(round(difyrel.mean(0),4))+'\n'+'Residuals standard deviation: '+str(round(difyrel.std(0),4)), fontsize=11)

fname = 'MLP-Histogram of residuals_y-ŷ_relative-'+str(neurons) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Residuals -> RELATIVE VALUES
plt.figure(5,figsize=(8,8))
plt.plot([0,2600],[0,0],'-.', color='grey')
plt.plot(difyrel,'k.')
plt.ylabel('(y-ŷ)/y')
plt.xlabel('Observation')
#plt.ylim((-0.13,0.13))
plt.title('Dispersion of the residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Relative values')
fname = 'MLP-Dispersion of the residuals MLP_relative-' +str(neurons)+ '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Print Results
print("r2_complete data: ", mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[0,2])
print("r_complete data: ", np.sqrt(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[0,2]))
print(" residuals mean and stdev: ", dify.mean(0),dify.std(0))
print(" Maximum and Minimum error: ", abs(dify).max(),abs(dify).min())
print(" Mean absolute error: ",sum(abs(dify))/dify.shape[0])

In [ ]:
# MSE: Mean Squared Error
mse_mlp=sum((mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])*(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1]))/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0].shape[0]
print(mse_mlp)

np.mean(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])
print(dify.mean(0))
print(dify.std(0))

print(np.mean((mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]),np.mean(dify/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]))


In [ ]:
neurons=12
# 4.2 Plot y vs ŷ -> TEST SET
plt.figure(1,figsize=(8,8))
plt.plot([73,120],[73,120],'-.', color='grey')
plt.plot(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0],mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1],'k.')
plt.xlabel('Y_test (SO2 real values)')
plt.ylabel('Ŷ_test (SO2 estimatives)')
plt.ylim((73,120))
plt.xlim((73,120))
plt.title('MLP regression avaliation: ŷ vs y '+"\n (" +str(neurons)+' units at hidden layer and tanh as activation function)')
plt.text(75,115,'R2 test: '+str(round(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[0,2],4))+'\n'+'R test: '+str(round(np.sqrt(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[0,2]),4)), fontsize=12)
fname = 'MLP-Regression_y vs ŷ ' +str(neurons)+ '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Histogram -> y-ŷ -> ABSOLUTE VALUES

dify=(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])
plt.figure(2,figsize=(8,8))
plt.hist(sorted(dify),50, density=True, color='gray')
plt.title('Histogram of residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Absolute values')
plt.xlabel('y-ŷ')
plt.ylabel('Probability density')
plt.text(-12,0.155,'Residuals mean: '+str(round(dify.mean(0),4))+'\n'+'Residuals standard deviation: '+str(round(dify.std(0),4)), fontsize=11)

fname = 'MLP-Histogram of residuals_y-ŷ_absolute-'+str(neurons) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Residuals -> ABSOLUTE VALUES
plt.figure(3,figsize=(8,8))
plt.plot([0,2600],[0,0],'-.', color='grey')
plt.plot(dify,'k.')
plt.ylabel('y-ŷ')
plt.xlabel('Observation')
plt.ylim((-13,13))
plt.title('Dispersion of the residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Absolute values')
fname = 'MLP-Dispersion of the residuals MLP_absolute-' +str(neurons)+ '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Histogram -> y-ŷ -> RELATIVE VALUES

dify=(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])
difyrel=dify/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]
plt.figure(4,figsize=(8,8))
plt.hist(sorted(difyrel),50, density=True, color='gray')
plt.title('Histogram of residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Relative values')
plt.xlabel('(y-ŷ)/y')
plt.ylabel('Probability density')
plt.text(-0.15,16,'Residuals mean: '+str(round(difyrel.mean(0),4))+'\n'+'Residuals standard deviation: '+str(round(difyrel.std(0),4)), fontsize=11)

fname = 'MLP-Histogram of residuals_y-ŷ_relative-'+str(neurons) + '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

#Residuals -> RELATIVE VALUES
plt.figure(5,figsize=(8,8))
plt.plot([0,2600],[0,0],'-.', color='grey')
plt.plot(difyrel,'k.')
plt.ylabel('(y-ŷ)/y')
plt.xlabel('Observation')
#plt.ylim((-0.13,0.13))
plt.title('Dispersion of the residuals - MLP: '+str(neurons)+' units at hidden layer'+'\n'+'Relative values')
fname = 'MLP-Dispersion of the residuals MLP_relative-' +str(neurons)+ '.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
# MSE: Mean Squared Error
#neurons = 12
mse_mlp=sum((mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])*(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1]))/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0].shape[0]
print(mse_mlp)

np.mean(mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])
print(dify.mean(0))
print(dify.std(0))

print(np.mean((mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]-mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,1])/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]),np.mean(dify/mlp_test.loc[idx[neurons,'tanh','lbfgs','constant']].iloc[:,0]))

print(work.columns)
print(work.shape)

## Sensitivity analysis

In [ ]:
#Variables Identification
listvar=data.columns[:-1]
data.columns

In [ ]:
# MULTIPLE LINEAR REGRESSION 
reg=sk.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
#Treinar o modelo com todos os dados
freg=reg.fit(xtrain,ytrain)
parr=freg.get_params(deep=True)
#Prediction
yhat=freg.predict(xtest) #Test Prediction
Yh=scalery.inverse_transform(yhat.reshape(-1,1)) #Return to original scale
R2test=freg.score(xtest,ytest) #Real test set
Yhat=np.reshape(Yh,-1)
#Mean Squared Error - Prediction
mse_mlr=sum((yte-Yhat)*(yte-Yhat))/yte.shape[0]

In [ ]:
#PREDICTION TRAINING SET
ytrhat=freg.predict(xtrain) #Test Prediction
ytrh=scalery.inverse_transform(ytrhat.reshape(-1,1)) #Return to original scale
R2tr=freg.score(xtrain,ytrain) #Real test set
Ytrhat=np.reshape(ytrh,-1)
#Mean Squared Error - Prediction
mse_mlrtrain=sum((ytr-Ytrhat)*(ytr-Ytrhat))/ytr.shape[0]

In [ ]:
### ---------  SENSITIVITY ANALYSIS  -------- ###
yMLR = {'Y':np.array(ytr),'yhat':Ytrhat}
#Coefficient of Determination
R2MLR={'r2train':R2tr}
#Mean Squared Error (MSE)
MSE_mlr={'MSEtrain':mse_mlrtrain}
#sum all deviations
sumdevR2MLR=0
#Mean of each variable
mean=xtrain.mean(axis=0)
stdev=xtrain.std(axis=0)
#Modifications in the input data: substitution of each variable for the its mean value
for i in range(len(listvar)):# -1 -> due to output    
    x1=xtrain.copy()
    stdev=xtrain.std(axis=0)
    
    #mst='m' #Disturbancy: Only mean
    x1[:250,i]=mean[i]+0.5*stdev[i]
    x1[250:500,i]=mean[i]+1*stdev[i]
    x1[500:750,i]=mean[i]+1.5*stdev[i]
    x1[750:,i]=mean[i]+2*stdev[i]
    mst='m+kstd' #Disturbancy: Mean+-k*stdev
    
    x1[:,i]=mean[i]
    #Prediction
    y1=freg.predict(x1)
    yh1=scalery.inverse_transform(y1.reshape(-1,1)) #Return to original scale
    Y1=np.reshape(yh1,-1)
    r21=freg.score(x1,ytrain) 
    #Mean Squared Error - Prediction
    mse_mlr1=sum((ytr-Y1)*(ytr-Y1))/ytr.shape[0]
    #Register
    yMLR.update({'yhat_mean_'+str(listvar[i]):Y1})
    R2MLR.update({'R2_mean_'+str(listvar[i]):r21})
    MSE_mlr.update({'MSE_mean_'+str(listvar[i]):mse_mlr1})
    sumdevR2MLR+=(R2tr-r21)
yMLR
MSE_mlr
#R2MLR

In [ ]:
MSEmlr=pd.DataFrame.from_dict(MSE_mlr,orient='index', columns=['MSE_MLR'])
MSEmlr.sort_values('MSE_MLR')

In [ ]:
#MEAN SQUARED ERROR CHART - MLR
MSEmlr=pd.DataFrame.from_dict(MSE_mlr,orient='index', columns=['MSE_MLR'])
msemlr = MSEmlr.sort_values('MSE_MLR',ascending=False)
plt.figure()
ms=msemlr.plot.bar(figsize=(10,7), title='MSE MLR',color='gray')
ms.set_ylabel('MSE')
for p in ms.patches:
    ms.annotate(str(round(p.get_height(),2)), (p.get_x() *0.995, p.get_height() * 1.005))

fname = 'MLR-Sensitivity.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
#Neural network Model
## 4.2 Multilayer Perceptron
#Number of hidden layers variation
N=30 #maximal number of neurons at hidden layer
kn=5 #number of splits at cross-validation procedure
t=1e-3 #Tolerance (modifications at r2 - > training the net)
niter=7
ns=9

nn=MLPRegressor(hidden_layer_sizes=(ns+1,),
                           activation='tanh', 
                           solver='lbfgs',
                           batch_size='auto',
                           learning_rate= 'constant',
                           max_iter=1000,
                           random_state=rs,
                           tol=t,
                           n_iter_no_change=niter,
                           verbose=False)
#Training with all data
rb=nn.fit(xtrain,ytrain)
#Test with the extra testing set - For each cross validation
yhat=nn.predict(xtest)
yh=yhat.reshape(-1,1)
R2=rb.score(xtest,ytest)
Yh=scalery.inverse_transform(yh)
Yhat=np.reshape(Yh,-1)
#Mean Squared Error - Prediction
mse_mlp=sum((yte-Yhat)*(yte-Yhat))/yte.shape[0]


In [ ]:
#PREDICTION TRAINING SET
ytrhat=nn.predict(xtrain) #Test Prediction
ytrh=scalery.inverse_transform(ytrhat.reshape(-1,1)) #Return to original scale
R2tr=nn.score(xtrain,ytrain) #Real test set
Ytrhat=np.reshape(ytrh,-1)
#Mean Squared Error - Prediction
mse_mlptrain=sum((ytr-Ytrhat)*(ytr-Ytrhat))/ytr.shape[0]


In [ ]:
### ---------  SENSITIVITY ANALYSIS  -------- ###
yMLP = {'Y':np.array(ytr),'yhat':Ytrhat}
#Coefficient of Determination
R2MLP={'Training':R2tr}
#Mean Squared Error (MSE)
MSE_mlp={'Training':mse_mlptrain}
#sum all deviations
sumdevR2MLP=0
#Mean of each variable - original range
meano=xtr.mean(axis=0)
stdevo=xtr.std(axis=0)
#Mean of each variable - scaled
mean=xtrain.mean(axis=0)
stdev=xtrain.std(axis=0)

#Modifications in the input data: substitution of each variable for the its mean value
for i in range(len(listvar)):# -1 -> due to output    
    x1=xtrain.copy()
    x1o=xtr.copy()
    #x1[:,i]=mean[i]

    #Disturbancy: Only mean - original range
    x1o.iloc[:250,i]=meano[i]+0.5*stdevo[i]
    x1o.iloc[250:500,i]=meano[i]+1*stdevo[i]
    x1o.iloc[500:750,i]=meano[i]+1.5*stdevo[i]
    x1o.iloc[750:,i]=meano[i]+2*stdevo[i]

    #mst='m' #Disturbancy: Only mean - Scaled
    x1[:250,i]=mean[i]+0.5*stdev[i]
    x1[250:500,i]=mean[i]+1*stdev[i]
    x1[500:750,i]=mean[i]+1.5*stdev[i]
    x1[750:,i]=mean[i]+2*stdev[i]
    mst='m+kstd' #Disturbancy: Mean+-k*stdev

    #Prediction
    y1=nn.predict(x1)
    yh1=scalery.inverse_transform(y1.reshape(-1,1)) #Return to original scale
    Y1=np.reshape(yh1,-1)
    r21=nn.score(x1,ytrain) 
    #Mean Squared Error - Prediction
    mse_mlp1=sum((ytr-Y1)*(ytr-Y1))/ytr.shape[0]
    #Register
    yMLP.update({'yhat_mean_'+str(listvar[i]):Y1})
    R2MLP.update({str(listvar[i]):r21})
    MSE_mlp.update({str(listvar[i]):mse_mlp1})
    sumdevR2MLP+=(R2tr-r21)

    #Plot Disturbances
    plt.figure(figsize=(10,7))
    plt.plot(x1o.reset_index().iloc[:,i+1],'k--',label='Disturbed')
    plt.plot(xtr.reset_index().iloc[:,i+1],'k',alpha=0.5,label='Not Disturbed')
    plt.legend()
    plt.title(x1o.columns[i])
    plt.ylabel(x1o.columns[i])
    fname = 'Disturbance'+str(x1o.columns[i][:-4])+'.png'
    plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

yMLP
MSE_mlp
#R2MLP

In [ ]:
x1o.columns[i]

In [ ]:
#Transform in DataFrame
MSEmlp=pd.DataFrame.from_dict(MSE_mlp,orient='index', columns=['MSE_MLP'])
R2mlp=pd.DataFrame.from_dict(R2MLP,orient='index', columns=['R2_MLP'])
Ymlp=pd.DataFrame.from_dict(yMLP)
MLP_results=MSEmlp.copy()
MLP_results['R2_MLP']=R2mlp.copy()
print(MLP_results)

'''
fname = '\Sensitivity_MLP.xlsx'
writer = pd.ExcelWriter(dir+fname, engine='openpyxl')

Ymlp.to_excel(writer,sheet_name='ymlp')
MLP_results.to_excel(writer,sheet_name='MSEr2')

        
writer.save()
writer.close()'''

In [ ]:
#MEAN SQUARED ERROR CHART
MSEmlp=pd.DataFrame.from_dict(MSE_mlp,orient='index', columns=['MSE_MLP'])
msemlp = MSEmlp.sort_values('MSE_MLP',ascending=False)
plt.figure()
mse=msemlp.plot.bar(figsize=(10,7), title='MSE MLP', color='gray')
plt.plot((0,14),(MSEmlp.iloc[0],MSEmlp.iloc[0]),'k--')
mse.set_ylabel('MSE')
for p in mse.patches:
    mse.annotate(str(round(p.get_height(),2)), (p.get_x() * 0.995, p.get_height() * 1.005))
    
fname = 'MLP-Sensitivity.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
#Ver
#https://www.python-course.eu/graphs_python.php
print(MSEmlp.iloc[1::,:]/MSEmlp.iloc[1::,:].min())
print(MSEmlp.iloc[1::,:]/MSEmlp.iloc[1::,:].sum()*100)

In [ ]:
#MEAN SQUARED ERROR CHART
MSEmlp=pd.DataFrame.from_dict(MSE_mlp,orient='index', columns=['MSE'])
#msemlp = MSEmlp.sort_values('MSE_MLP',ascending=False)
plt.figure()
mse=MSEmlp.plot.bar(figsize=(10,7), title='MSE MLP', color='gray',alpha=0.7)
plt.plot((0,15),(MSEmlp.iloc[0],MSEmlp.iloc[0]),'k--',alpha=0.7)
mse.set_ylabel('Erro quadrático médio (MSE)')
for p in mse.patches:
    mse.annotate(str(round(p.get_height(),3)), (p.get_x() * 0.995, p.get_height() * 1.005))
plt.annotate('MSE referência = '+str(round(MSEmlp.iloc[0][0],3))+'ppm',(7,0.5*MSEmlp.iloc[0]))
    
fname = 'MLP-Sensitivity-desorder.png'
plt.savefig(os.path.join(dir, fname), bbox_inches='tight', format='png', dpi=600)

In [ ]:
#VAriables that most inffluence
varinf=['13-F_TA_T/H', '7-F_PA_T/H', '1-F_FUEL_T/H','10-F_SA_T/H']
ooo=['18-C_SO2_PPM']
for i in range(len(varinf)):
    plt.figure(figsize=(10,7))
    plt.plot(work[varinf[i]],work[ooo],'k.')
    plt.ylabel(ooo[0])
    plt.xlabel(varinf[i])
    '''
    plt.figure(figsize=(10,7))
    plt.plot(work[ooo],'k',label=ooo[0])
    plt.plot(work[varinf[i]],'b',label=varinf[i])
    plt.ylabel('Values')
    plt.xlabel('Time serie')
    plt.legend()
    '''
    fig = plt.figure(figsize=(10,7))
    ax1 = fig.add_subplot(111)
    
    ax2 = ax1.twinx()
    ax1.plot(work[ooo],'k',alpha=0.6,label=ooo[0])
    ax2.plot(work[varinf[i]],'k-.',alpha=1,label=varinf[i])

    ax1.set_xlabel('X data')
    ax1.set_ylabel(str(ooo[0])+' data', color='k')
    ax2.set_ylabel(str(varinf[i])+' data', color='k')
    ax2.set_ylim(work[varinf[i]].min()*0.9,work[varinf[i]].max()*1.01)
    fig.legend()

In [ ]:
teste=work[varinf]
teste[ooo]=work[ooo]

#CORRELATION MATRIX
corw=teste.corr() #Correlation Matrix
corw.style.background_gradient(cmap='PuBu')#'coolwarm')

###  Fim